## Problem Set 03
### AST 8110, Fall 2022
##### Tyler Barna

In [1]:
## Importing packages (some of these may not be useful)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import time
#bigstart = time.time()
#import numba

import sys
import os
import astropy
import astropy.units as u
import astropy.constants as const
import astropy.coordinates as coord
import random

from tqdm import tqdm
from scipy.stats import binned_statistic as binstat
from scipy import integrate

## Set Seed
random.seed(8110)


All quantities below are assumed to be project on the plane of the sky.

The mass distribution in the lens is assymmetric, centered on $(0,0)$, and its raidal density profile is given by
\begin{equation}
\kappa = \kappa_{0} \left( \frac{|\vec{\theta}| + \theta_{0}}{\theta_{0}} \right)^{m}
\end{equation}
where $\kappa_{0}=30$, $\theta_{0}=1$, and $m=-3.2$. The profile is truncated at $\theta_{max}=0.5$ such that $\kappa(\theta>\theta_{max})=0$.

A source is placed on the positive $\theta_{x}$ axis at $\theta_{x}=\beta=2$ from the origin. The source is a point source with a flux of $F_{0}=1$.Because of symmetry, all images will be located on the $\theta_{x}$ axis.

## (a) Time Delay Surface

\begin{equation}
T\left( \theta_{x}\right) \propto
\frac{1}{2} \left( \theta_{x} - \beta \right)^{2}
- \frac{1}{\pi} \int \text{d} \theta_{x}^{\prime} \text{d} \theta_{y}^{\prime} \kappa\left( \theta_{x}^{\prime} , \theta_{y}^{\prime}  \right)
\text{ln} \sqrt{\left( \theta_{x} - \theta_{x}^{\prime} \right)^{2} + \left( \theta_{y}^{\prime} \right)^{2}}

\end{equation}

## (b) Lens Equation 

The lens equation can be written as

\begin{equation}
\vec{\theta} - \vec{\beta} = \vec{\alpha}
\end{equation}

where $\vec{\alpha}$ is the deflection angle. 

The magnitude of the deflection angle can be calculated as

\begin{equation}
\alpha = \frac{M\left( \leq \theta \right)}{\pi \theta}
\end{equation}

where $M\left( \leq \theta \right)$ is the mass enclosed within a radius $\theta$ of the origin.